In [ ]:
from __future__ import print_function 

%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1*X GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024*4))])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)



import pandas as pd
import pickle
import gc, re, copy
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

# Project imports 
from data.tasks import *
from data.label_shift import label_shift_linear, plot_labeldist, plot_splitbars
from experiments.models import *
from experiments.training import *
from bounds.bounds import *
from util.kl import *
from util.misc import *
from results.plotting import *

# Hyper-parameters
batch_size = 128
 

TASK = 4

# Load and structure data

In [ ]:
x_source,y_source,x_target,y_target=load_task(TASK)

# Functions for evaluating bound

# Run evaluation 

In [ ]:
from bounds.bounds import *
from util.batch import *
architecture="lenet"
delta=0.05
Binary=True
epsilons=[0.01]
alphas=[0.3]#0,0.3]
sigmas=[[3,2]]

for alpha in alphas:
    print("alpha:"+str(alpha))
    if alpha==0:
        x_bound=x_source
        y_bound=y_source
    else:
        x_bound, x_prior, y_bound , y_prior = train_test_split(x_source,y_source,test_size=alpha,random_state=69105)
        #del x_source, y_source
        #x_bound=x_bound[:1000]
        #y_bound=y_bound[:1000]
        #x_target=x_target[:1000]
        #y_target=y_target[:1000]
    for epsilon in epsilons:
        print("epsilon:"+str(epsilon))
        for sigma in sigmas:    
            print("sigma:"+str(sigma))
            paths=posterior_checkpoints(TASK, epsilon, alpha, binary=True,architecture="lenet")
            for p in paths:
                results=compute_bound_parts(TASK,p,x_bound[:1000],y_bound[:1000],x_target[:1000],y_target[:1000],alpha=alpha,sigma=sigma,epsilon=epsilon,binary=Binary,architecture=architecture)
               #task, posterior_path, x_bound, y_bound, x_target, y_target, alpha=0.1, delta=0.05, epsilon=0.01, 
                  #prior_path=None, bound='germain', binary=False, n_classifiers=4, sigma=[3,3], seed=None,batch_size=128,architecture="lenet"):